# Random Forest Regression

This notebook is based on [NN_SVM_RF_classification_supervised_Evaluated.ipynb](https://github.com/RichardsGroup/AGN_DataChallenge/blob/main/submissions/SER-SAG/NN_SVM_RF_classification_supervised_Evaluated.ipynb) submission to the LSST AGN Data Challenge by SER-SAG.

Authors: Djordje Savic (Postdoc), Isidora Jankov (PhD student), Iva Čvorović-Hajdinjak (PhD student)

## Data reading, selection and pre-processing

In [1]:
## commonly used modules
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import os, sys
import yaml
import seaborn as sns
import importlib

from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets 
from sklearn import preprocessing

from sklearn.ensemble import RandomForestRegressor
import sklearn.model_selection as model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from sklearn.metrics import f1_score
#from sklearn.metrics import plot_confusion_matrix #command is deprecated
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

pd.set_option('display.max_columns', 1000)

### Data selection
The object table follows the current version of the LSST Data Products document (LSE-163) as much as possible with measurements of included objects in the following main catalogries:
- __Astromety__ -> ra, dec, proper motion and parallax
- __Photometry__ -> point and extended source photometry, in both AB magnitdues and fluxes (nJy)
- __Color__ -> Computed using the fluxes
- __Morphology__ -> 1 for extended and 0 for point-like
- __Light Curve Features__ -> Extrated on the SDSS light curves if matched
- __Redshift__ -> Both spectroscopic and photometric, wherever available

The data used here is from the 's82ObjectTable.parquet' file available online on [Zenodo](https://zenodo.org/records/6862159).

In [2]:
sample1 = pd.read_csv('select_attributes1.csv', index_col=0)

In [3]:
sample1

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
271386,2.047289,1.542644,1.517730,0.691103,0.180547,Star,0.000000
271388,2.628649,1.437541,0.672900,0.403049,NaN,Star,0.000000
271389,0.506060,-0.086831,-0.137316,-0.160921,NaN,Star,0.000000
271390,1.318985,0.514469,0.154915,0.077669,-0.012270,Star,0.000000
271391,0.964024,0.301733,0.076412,0.028065,-0.041763,Star,0.000000
...,...,...,...,...,...,...,...
1468017,0.451338,0.154067,0.368441,0.338225,NaN,Qso,0.442569
1468018,0.065456,0.106526,-0.103106,-0.005643,NaN,Qso,1.110641
1468019,0.156834,0.123606,0.331176,0.023310,NaN,Qso,1.775506


In [4]:
# Changing the class value to numerical for using sparse categorical cross entropy loss from tensorflow
# Also moving Agn and highZQso to Qso label
sample1_good = sample1.replace({'class': {'Star': 0, 'Gal': 1, 'Qso': 2, 'Agn': 2, 'highZQso': 2}})
sample1_good

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
271386,2.047289,1.542644,1.517730,0.691103,0.180547,0.0,0.000000
271388,2.628649,1.437541,0.672900,0.403049,NaN,0.0,0.000000
271389,0.506060,-0.086831,-0.137316,-0.160921,NaN,0.0,0.000000
271390,1.318985,0.514469,0.154915,0.077669,-0.012270,0.0,0.000000
271391,0.964024,0.301733,0.076412,0.028065,-0.041763,0.0,0.000000
...,...,...,...,...,...,...,...
1468017,0.451338,0.154067,0.368441,0.338225,NaN,2.0,0.442569
1468018,0.065456,0.106526,-0.103106,-0.005643,NaN,2.0,1.110641
1468019,0.156834,0.123606,0.331176,0.023310,NaN,2.0,1.775506


In [5]:
# dropping nan values
sample1_good = sample1_good.dropna()

display(sample1_good.columns, sample1_good.describe())

Index(['stdColor_0', 'stdColor_1', 'stdColor_2', 'stdColor_3', 'stdColor_4',
       'class', 'z'],
      dtype='object')

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
count,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000,251092.000000
mean,1.192455,0.796617,0.458105,0.246363,0.137506,0.969585,0.699422
std,1.097236,0.577364,0.350144,0.179869,0.181409,0.717093,0.706082
min,-3.456974,-4.951052,-8.632828,-2.787663,-3.639812,0.000000,-0.006079
25%,0.383977,0.338318,0.165244,0.097926,0.010907,0.000000,0.000000
50%,0.989902,0.611921,0.444655,0.257902,0.111104,1.000000,0.602561
75%,1.892795,1.360131,0.669739,0.372838,0.225287,1.000000,0.925258
max,16.439950,8.568064,2.936810,4.250202,2.608742,2.000000,7.011245


### Data preprocessing 

In [6]:
# choose the records with class = 1 (galaxies)
only_gal = sample1_good[sample1_good['class']==1.0]
only_gal

,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class,z
objectId,,,,,,,
696229,0.199562,1.583992,0.931038,0.405977,0.115194,1.0,0.607717
696230,0.613735,0.259783,0.597732,0.238398,0.445753,1.0,0.981158
696233,1.494033,0.746691,0.360383,0.090811,0.347208,1.0,0.206798
696234,2.323856,1.572886,0.706285,0.335051,0.368864,1.0,0.477364
696236,1.471431,1.798605,0.716381,0.389229,0.044622,1.0,0.492501
...,...,...,...,...,...,...,...
968007,0.711156,0.483807,0.652757,0.215973,0.170660,1.0,0.863240
968009,-0.158420,0.639037,0.843015,0.268124,-0.101792,1.0,0.919322
968010,-0.058365,0.561562,0.651556,0.209875,0.672189,1.0,0.989209


In [7]:
#attr_list = ['stdColor_0','stdColor_1','stdColor_2','stdColor_3','stdColor_4','class','z']

In [8]:
#we need to drop Na values from all columns
#print(len(only_gal))

#for attr in attr_list:
    #print(attr)
#    only_gal = only_gal.dropna(subset=[attr])

#print(len(only_gal))

In [9]:
only_gal.to_csv('only_gal_good.csv')

In [2]:
only_gal = pd.read_csv('only_gal_good.csv')

In [3]:
print(len(only_gal))

121743


In [4]:
#half the sample to reduce computational time
only_gal = only_gal[:int((len(only_gal)-1)/4)]

In [5]:
print(len(only_gal))

30435


In [6]:
#splitting into X and y values

X = only_gal
# removing the z attribute from X data array
X = X.drop(['z'], axis=1)

# storing labels for later
y = only_gal['z']

display(X.shape, y.shape)
display(X.describe()) #no z property

(30435, 7)

(30435,)

,objectId,stdColor_0,stdColor_1,stdColor_2,stdColor_3,stdColor_4,class
count,30435.000000,30435.000000,30435.000000,30435.000000,30435.000000,30435.000000,30435.0
mean,729868.928569,1.231604,1.002493,0.645353,0.337394,0.302180,1.0
std,20234.882955,1.126895,0.542076,0.211590,0.118605,0.204329,0.0
min,696229.000000,-1.843920,-0.788874,-0.646829,-1.156918,-0.813586,1.0
25%,712244.500000,0.441856,0.538733,0.508076,0.260688,0.185036,1.0
50%,729269.000000,0.914732,0.772054,0.624316,0.334100,0.276960,1.0
75%,747167.500000,1.934255,1.555041,0.759421,0.411571,0.388612,1.0
max,766616.000000,16.439950,2.399798,1.581597,1.146042,1.577499,1.0


In [7]:
## Splitting data to train, test followed by standardization
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.5, random_state = 1)

## standardization using StandardScaler applied to X_train (yields mean and sigma for X_train)
## and then standardizing X_train and X_test with mean and sigma obtained from X_train

scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

display(X_train.shape, X_test.shape)

(15217, 7)

(15218, 7)

## Random forest regression
Create a random forest classifier.

In [8]:
clf=RandomForestRegressor(n_estimators=2001, n_jobs=70, criterion="squared_error")

Source: [scikit-learn](https://scikit-learn.org/dev/modules/generated/sklearn.ensemble.RandomForestRegressor.html)

In [9]:
#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=2001, n_jobs=70)

In [10]:
y_pred_rf=clf.predict(X_test)

In [11]:
display(f"MSE: {mean_squared_error(y_test, y_pred_rf)}")
display(f"MAE: {mean_absolute_error(y_test, y_pred_rf)}")
display(f"R2 score: {r2_score(y_test, y_pred_rf)}")

'MSE: 0.00880556489319356'

'MAE: 0.05350057322266101'

'R2 score: 0.869411760434319'

R^2 score (coefficient of determination) represents how well predictions of a regression model fit the actual data.